In [6]:
import numpy as np
import wfdb
import glob
import matplotlib.pyplot as plt
from matplotlib.image import imread
import pywt
import pywt.data
import os
from pathlib import Path
from PIL import Image,ImageEnhance
from scipy import fftpack
import shutil
import pandas as pd
import random
import scipy.signal as signal
from wfdb import processing
from itertools import combinations
import ast
import wfdb
#from scipy.signal.filter_design import zpk2sos

Extract Lead v1 from the 12-Leads ECG Dataset

### 먼저 모든 데이터 한 파일에 몰아넣음

In [22]:
def Get_LeadV1_Numpy():

    for myloop in range(1, 21839, 1):
        five_digit_value = '%05d' % myloop
        signals, fields = wfdb.rdsamp("/home/ines/code/ptb-xl/data/" + five_digit_value + "_lr", channels=[6],sampto=1000)

        signals = signals.tolist()
        # print(signals)

        newest = [i[0] for i in signals]
        X = [i for i in newest if i != 0.0]
        X = np.array(X)
        np.save("/home/ines/code/ptb-xl/Dest/" + five_digit_value + '.npy', X)  # save

Get_LeadV1_Numpy()

FileNotFoundError: [Errno 2] No such file or directory: '/home/ines/code/ptb-xl/data/21838_lr.hea'

Numpy Files distruction to classes (with the help of CSV file)

Denoising Numpy Files

In [23]:
def Numpy_Denoise():

    # 신호의 평균 절대 편차를 계산
    def madev(d, axis=None):
        """ Mean absolute deviation of a signal """
        return np.mean(np.absolute(d - np.mean(d, axis)), axis)

    # 웨이블릿 변환을 사용하여 신호에서 노이즈를 제거하는 함수
    # 쌍직교 함수
    # 레벨1로 웨이블릿 변환
    def wavelet_denoising(x, wavelet='bior3.1', level=1):
        # coeff: 웨이블릿 변환을 통해 계산된 근사 계수와 세부 계수를 저장하는 변수
        coeff = pywt.wavedec(x, wavelet, mode="per")
        # 세부 계수의 평균 절대 편차를 계산하여 노이즈의 크기를 추정하는 데 사용
        sigma = (1 / 0.6745) * madev(coeff[-level])
        # uthresh: 임계값으로, 노이즈의 크기를 기반으로 계산
        uthresh = sigma * np.sqrt(2 * np.log(len(x)))
        coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
        # Multilevel reconstruction using waverec
        # wavelet: Wavelet object or name string
        return pywt.waverec(coeff, wavelet, mode='per')

    # 20446
    src_dir = "/home/ines/code/ptb-xl/source/"
    dest_dir = "/home/ines/code/ptb-xl/dest/"

    # 반복문을 통해 모든 .npy 파일에 대해 노이즈 제거를 수행합니다. 각 파일을 웨이블릿 변환을 통해 필터링하고 새로운 파일로 저장
    for file in glob.iglob(src_dir + '**/*.npy', recursive=True):
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:5]

        signal = np.load(file)

        # wavelist=['bior1.3','bior2.8','bior3.1','bior3.9','db2','db8','rbio1.5','sym6','sym8']
        wavelist = ['bior3.1']


        for wav in wavelist:
            filtered = wavelet_denoising(signal, wavelet=wav, level=1)
            np.save(dest_dir+new_file_name+'.npy', filtered) # save

Numpy_Denoise()

Frequency Filtration to reduce data size

In [2]:
def Freq_Filteration():
    #### https://scipy-lectures.org/intro/scipy/auto_examples/plot_fftpack.html

    src_dir = "/home/ines/code/ptb-xl/source_denoise/"
    dest_dir = "/home/ines/code/ptb-xl/source_filter/"

    for file in glob.iglob(src_dir + '**/*.npy', recursive=True):

        #Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:7]

        #Load the file
        x = np.load(file)  # 17876_lr

        # The FFT of the signal
        sig_fft = fftpack.fft(x)

        # The corresponding frequencies
        sample_freq = fftpack.fftfreq(x.size, d=0.02)

        #The original signal copy
        high_freq_fft = sig_fft.copy()
        high_freq_fft = np.delete(high_freq_fft, np.where(np.abs(sample_freq > 2)))

        #Taking Inverse of FFT
        filtered_sig = fftpack.ifft(high_freq_fft)
        print(len(filtered_sig))

        #Saving resultant filtered signal into npy
        np.save(dest_dir+new_file_name+".npy", filtered_sig)
    
Freq_Filteration()

541
537
528
536
538
541
539
533
541
535
535
541
538
535
541
541
525
538
532
534
537
532
530
539
531
538
541
537
532
537
536
537
537
535
539
519
531
538
541
537
536
528
541
533
537
530
538
539
539
541
538
536
530
537
535
538
537
537
538
539
538
539
541
522
537
533
539
539
537
538
538
533
539
541
539
541
539
534
536
541
537
532
534
529
538
539
541
534
539
539
537
539
541
541
536
537
538
538
535
536
539
541
537
531
535
530
537
532
538
539
530
541
533
538
530
533
539
538
526
535
533
516
538
541
539
537
533
533
539
538
541
537
538
535
528
531
541
536
535
536
534
537
539
535
537
539
541
535
541
533
541
539
538
541
537
529
538
538
535
541
533
529
534
535
539
536
539
528
538
536
537
539
539
538
536
533
539
537
535
523
536
532
537
537
537
533
538
535
538
541
537
528
538
538
541
536
538
538
538
531
537
538
532
541
537
537
538
538
539
537
538
541
538
536
539
541
537
506
539
531
535
529
535
541
536
539
538
541
534
530
526
534
538
539
518
541
538
537
539
539
535
537
535
538
541
539
529
528
538
532


Spectrograms Using STFT (Scipy Library)

### 한 번에 돌리기엔 너무 많아서 따로 파일 두고 8000개 씩 돌림

In [42]:
def Draw_Spectrogram():
    
    src_dir = "/home/ines/code/ptb-xl/source_filter/3000/"
    dest_dir = "/home/ines/code/ptb-xl/source_spec/"


    for file in glob.iglob(src_dir + '**/*.npy', recursive=True):
        # Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:7]   

        #Loading file
        x = np.load(file)   
        print(len(x))

        #applying STFT transformation
        f, t, Zxx = signal.stft(x, nfft=9, nperseg=9, noverlap=5, window='hann') #NFFT calculate by N base Log 2 with x=len(x) 
        plt.figure(figsize=(1.15, 1.19))   

        #plotting spectrogram
        plt.pcolormesh(t, f, np.abs(Zxx), cmap="gray_r")
        plt.axis("off")
        plt.savefig(dest_dir+ new_file_name + '.png',bbox_inches='tight',pad_inches=0)
        plt.close('all')

Draw_Spectrogram()

538


/tmp/ipykernel_130760/2464692961.py:17: UserWarning: Input data is complex, switching to return_onesided=False
  f, t, Zxx = signal.stft(x, nfft=9, nperseg=9, noverlap=5, window='hann') #NFFT calculate by N base Log 2 with x=len(x)
/tmp/ipykernel_130760/2464692961.py:21: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  plt.pcolormesh(t, f, np.abs(Zxx), cmap="gray_r")


Raw Signal Graph

# figsize 수정 필요

In [24]:
# fig size 수정 필요

def DrawGraphs():

    src_dir = "/home/ines/code/ptb-xl/source_filter/3000/"
    dest_dir = "/home/ines/code/ptb-xl/source_gr"

    for file in glob.iglob(src_dir + '**/*.npy', recursive=True):
        # Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:7]  

        # Loading file
        x = np.load(file)   
        print(len(x))

        plt.figure(figsize=(10, 10))   
        plt.plot(x, 'gray')
        plt.axis("off")

        # saving plots
        plt.savefig(dest_dir + new_file_name + '.jpeg')
        plt.close('all')

        
DrawGraphs()

539


/home/ines/anaconda3/envs/jyp/lib/python3.9/site-packages/matplotlib/cbook.py:1699: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/home/ines/anaconda3/envs/jyp/lib/python3.9/site-packages/matplotlib/cbook.py:1345: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


Two Data Augmentation Approaches on Spectrogram / Raw Signal Images

In [29]:
def HorizontalFlip():
    src_dir = "/home/ines/code/ptb-xl/source_spec/"
    dest_dir = "/home/ines/code/ptb-xl/source_flip/"

    for file in glob.iglob(src_dir + '**/*.png', recursive=True):
        # Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:5]
        print(file)
        #opening Image and convert into 2-D
        img = Image.open(file)
        img = np.array(img)

        # Flipping images with Numpy
        flipped_img = np.fliplr(img)

        img=Image.fromarray(flipped_img)
        img.save(dest_dir+new_file_name+"_hFlip"+".png")
        plt.close('all')

def SetContrast():
    src_dir = "/home/ines/code/ptb-xl/source_spec/"
    dest_dir = "/home/ines/code/ptb-xl/source_const/"

    for file in glob.iglob(src_dir + '**/*.png', recursive=True):
        # Extracting File Name
        existing_file_name = Path(file).stem
        new_file_name = existing_file_name[:5]

        # read the image
        im = Image.open(file)

        # image brightness enhancer
        enhancer = ImageEnhance.Contrast(im)

        factor = 1.7  # increase contrast
        im_output = enhancer.enhance(factor)
        im_output.save(dest_dir + new_file_name + "_Contrast" + ".png")


HorizontalFlip()
SetContrast()

/home/ines/code/ptb-xl/source_spec/21406.png
/home/ines/code/ptb-xl/source_spec/07392.png
/home/ines/code/ptb-xl/source_spec/02751.png
/home/ines/code/ptb-xl/source_spec/20770.png
/home/ines/code/ptb-xl/source_spec/14194.png
/home/ines/code/ptb-xl/source_spec/17108.png
/home/ines/code/ptb-xl/source_spec/01553.png
/home/ines/code/ptb-xl/source_spec/21277.png
/home/ines/code/ptb-xl/source_spec/21227.png
/home/ines/code/ptb-xl/source_spec/04644.png
/home/ines/code/ptb-xl/source_spec/08811.png
/home/ines/code/ptb-xl/source_spec/03731.png
/home/ines/code/ptb-xl/source_spec/18844.png
/home/ines/code/ptb-xl/source_spec/04082.png
/home/ines/code/ptb-xl/source_spec/19783.png
/home/ines/code/ptb-xl/source_spec/15258.png
/home/ines/code/ptb-xl/source_spec/16149.png
/home/ines/code/ptb-xl/source_spec/02828.png
/home/ines/code/ptb-xl/source_spec/05420.png
/home/ines/code/ptb-xl/source_spec/14989.png
/home/ines/code/ptb-xl/source_spec/05421.png
/home/ines/code/ptb-xl/source_spec/00237.png
/home/ines

### X_test, X_train 파일 생성 후 라벨 파일(자동 생성)에 학습, 테스트 용 데이터 분류함

- 문제점: label이 여럿으로 뜸

ex) ['HYP', 'CD', 'MI']

In [52]:
import os
import shutil
import pandas as pd


def distribute_images():
    test_fold = 10
    y_train = {}
    y_test = {}
    
    # 데이터베이스(csv 파일) 로드
    df_db = pd.read_csv('/home/ines/code/ptb-xl/ptbxl_database.csv', index_col='ecg_id')
    df_db.scp_codes = df_db.scp_codes.apply(lambda x: ast.literal_eval(x))
    df_st = pd.read_csv('/home/ines/code/ptb-xl/scp_statements.csv', index_col=0)
    df_st = df_st[df_st.diagnostic == 1]

    def aggregate_diagnostic(y_dic):
        tmp = []
        for key in y_dic.keys():
            if key in df_st.index:
                tmp.append(df_st.loc[key].diagnostic_class)
        return list(set(tmp))
    
    # Apply diagnostic superclass
    df_db['diagnostic_superclass'] = df_db.scp_codes.apply(aggregate_diagnostic)
    

    # 이미지 파일(png) 및 레이블 식별 및 훈련/테스트 세트로 분배
    for i in range(len(df_db)):
        five_digit_value = '%05d' % i
        filee_name = df_db.iloc[i][0]
        five_digit_value = '%05d' % int(filee_name)
        file_name = five_digit_value + ".png"  # 이미지 파일 이름
        class_name = df_db.iloc[i].diagnostic_superclass          # 레이블(class) 이름

        source_dir = "/home/ines/code/ptb-xl/source_spec/" + file_name

        train_dir = '/home/ines/code/ptb-xl/X_train/' + str(class_name) + '/'
        test_dir = '/home/ines/code/ptb-xl/X_test/' + str(class_name) + '/'

        if df_db.iloc[i].strat_fold != 10:
            if not os.path.exists(train_dir):
                os.makedirs(train_dir)
            
            # 이미지 파일 이동 (파일이 이미 존재하지 않는 경우에만 이동)
            destination_path = os.path.join(train_dir, file_name)
            if not os.path.exists(source_dir):
                print(f"Warning: {source_dir} does not exist.")
            elif not os.path.exists(destination_path):
                shutil.move(source_dir, destination_path)
                y_train[five_digit_value] = df_db.iloc[i].diagnostic_superclass
            else:
                print(f"Warning: {destination_path} already exists.")
            
        else:
            if not os.path.exists(test_dir):
                os.makedirs(test_dir)
            
            # 이미지 파일 이동 (파일이 이미 존재하지 않는 경우에만 이동)
            destination_path = os.path.join(test_dir, file_name)
            if not os.path.exists(source_dir):
                print(f"Warning: {source_dir} does not exist.")
            elif not os.path.exists(destination_path):
                shutil.move(source_dir, destination_path)
                y_test[five_digit_value] = df_db.iloc[i].diagnostic_superclass
            else:
                print(f"Warning: {destination_path} already exists.")

    return y_train, y_test
            

# 이미지 분배 함수 호출
y_train, y_test = distribute_images()

/tmp/ipykernel_130760/109366221.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  filee_name = df_db.iloc[i][0]


In [55]:
# 결과 출력
print("y_train:", len(y_train))
print("y_test:", len(y_test))

y_train: 16017
y_test: 1814


In [ ]:

'''
import os
import shutil
import pandas as pd

test_fold = 10

def distribute_images():
    # 데이터베이스(csv 파일) 로드
    df_db = pd.read_csv('/home/ines/code/ptb-xl/ptbxl_database.csv', index_col='ecg_id')
    df_db.scp_codes = df_db.scp_codes.apply(lambda x: ast.literal_eval(x))
    df_st = pd.read_csv('/home/ines/code/ptb-xl/scp_statements.csv', index_col=0)
    df_st = df_st[df_st.diagnostic == 1]

    def aggregate_diagnostic(y_dic):
        tmp = []
        for key in y_dic.keys():
            if key in df_st.index:
                tmp.append(df_st.loc[key].diagnostic_class)
        return list(set(tmp))
    
    # Apply diagnostic superclass
    df_db['diagnostic_superclass'] = df_db.scp_codes.apply(aggregate_diagnostic)
    print(df_db.strat_fold)
    

    # 이미지 파일(png) 및 레이블 식별 및 훈련/테스트 세트로 분배
    for i in range(0, 21383, 1):
        filee_name = df_db.iloc[i][0]
        print(filee_name)
        five_digit_value = '%05d' % int(filee_name)
        file_name = five_digit_value + ".png"  # 이미지 파일 이름
        class_name = df_db.iloc[i].diagnostic_superclass          # 레이블(class) 이름

        source_dir = "/home/ines/code/ptb-xl/source_spec/" + file_name
        search_dir = '/home/ines/code/ptb-xl/dest/' + str(class_name) + '/'
        dest_dir = search_dir

        train_dir = '/home/ines/code/ptb-xl/X_train/' + str(class_name) + '/'
        test_dir = '/home/ines/code/ptb-xl/X_test/' + str(class_name) + '/'

        if df_db.iloc[i].strat_fold != 10:
            if not os.path.exists(X_train):
                os.makedirs(X_train)
            
            # 이미지 파일 이동
            shutil.move(source_dir, X_train)
            y_train = df_st[(df_st.strat_fold != test_fold)].diagnostic_superclass
            
        else:
            if not os.path.exists(X_test):
                os.makedirs(X_test)
            
            # 이미지 파일 이동
            shutil.move(source_dir, X_test)
            y_test = df_st[df_st.strat_fold == test_fold].diagnostic_superclass
            
        X_train = df_db[np.where(df_st.strat_fold != test_fold)]
        y_train = df_st[(df_st.strat_fold != test_fold)].diagnostic_superclass
        X_test = df_db[np.where(df_st.strat_fold == test_fold)]
        y_test = df_st[df_st.strat_fold == test_fold].diagnostic_superclass
        # 이미지 파일이 존재하는지 확인
        if not os.path.exists(source_dir):
            print(source_dir + " not found.")
        else:
            # 클래스별 디렉토리 생성
            if not os.path.exists(search_dir):
                os.makedirs(search_dir)
            
            # 이미지 파일 이동
            shutil.move(source_dir, dest_dir)

# 이미지 분배 함수 호출
distribute_images()
'''

In [ ]:
# py 파일 참고
# 마지막단 함수 구현 --> 라벨링 복수형 괜찮은지?
# train test split


# png파일 화질